# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started. Once you have completed the code you can download the notebook for making a submission.

In [1]:
import numpy as np
import os
from imageio import imread
from skimage.transform import resize
import datetime
import os

We set the random seed so that the results don't vary drastically.

In [2]:
np.random.seed(30)
import random as rn
rn.seed(30)
from tensorflow import keras
import tensorflow as tf
tf.random.set_seed(30)

In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

**data path: /home/datasets/Project_data**

In [3]:
train_doc = np.random.permutation(open('/home/datasets/Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('/home/datasets/Project_data/val.csv').readlines())
batch_size = 32
print(f'Training data size = {len(train_doc)} and Validation data size = {len(val_doc)}')

Training data size = 663 and Validation data size = 100


## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [4]:
img_size = (100,100)
def processImage(image, img_size=(64,64)):
    return resize(image, img_size, anti_aliasing=True)

In the below generator:
1. I am using all images (after trying out the reduced set).
2. All images are resized to 100x100
3. The last partial batch is handled with a condition and code is not repeated.

In [5]:
#img_idx = [0,1,2,3,4,6,8,10,12,14,16,18,20,22,24,25,26,27,28,29]
img_idx = [i for i in range(30)]

def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    x = len(img_idx)
    while True:
        t = np.random.permutation(folder_list)
        if len(t) % batch_size > 0:
            num_batches = int(len(t) / batch_size) + 1
        else:
            num_batches = int(len(t) / batch_size)
        y = img_size[0]
        z = img_size[1]
        for batch in range(num_batches): # we iterate over the number of batches
            current_batch_size = batch_size
            if batch == num_batches-1:
                current_batch_size = len(t) - batch*batch_size
            batch_data = np.zeros((current_batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((current_batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(current_batch_size): # iterate over the current_batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*current_batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*current_batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    
                    batch_data[folder,idx,:,:,0] = processImage(image[:,:,0], img_size=img_size)
                    batch_data[folder,idx,:,:,1] = processImage(image[:,:,1], img_size=img_size)
                    batch_data[folder,idx,:,:,2] = processImage(image[:,:,2], img_size=img_size)
                    
                batch_labels[folder, int(t[folder + (batch*current_batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [6]:
curr_dt_time = datetime.datetime.now()
train_path = '/home/datasets/Project_data/train'
val_path = '/home/datasets/Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 10# choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 10


## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

#### Model 1: Initial Model

In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GRU, Dropout, Flatten, BatchNormalization, Activation, Conv3D, MaxPooling3D
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras import optimizers

input_shape = (len(img_idx), img_size[0], img_size[1], 3)

#write your model here
model1 = Sequential()
model1.add(Conv3D(16, kernel_size=(3,3,3), activation='relu', input_shape=input_shape))
model1.add(MaxPooling3D(pool_size=(2, 2, 2)))

model1.add(Conv3D(32, kernel_size=(3,3,3), activation='relu'))
model1.add(MaxPooling3D(pool_size=(2, 2, 2)))

model1.add(Flatten())

model1.add(Dense(64, activation='relu'))
model1.add(Dense(5, activation='softmax'))

model1.summary()



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d (Conv3D)             (None, 28, 98, 98, 16)    1312      
                                                                 
 max_pooling3d (MaxPooling3D  (None, 14, 49, 49, 16)   0         
 )                                                               
                                                                 
 conv3d_1 (Conv3D)           (None, 12, 47, 47, 32)    13856     
                                                                 
 max_pooling3d_1 (MaxPooling  (None, 6, 23, 23, 32)    0         
 3D)                                                             
                                                                 
 flatten (Flatten)           (None, 101568)            0         
                                                                 
 dense (Dense)               (None, 64)                6

2024-09-03 11:04:46.845531: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
2024-09-03 11:04:46.845597: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 14800 MB memory:  -> device: 0, name: Quadro RTX 5000, pci bus id: 0000:3d:00.0, compute capability: 7.5


Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [8]:
optimiser = optimizers.Adam(lr=0.001, decay=1e-6)
model1.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy', keras.metrics.AUC()])
model1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d (Conv3D)             (None, 28, 98, 98, 16)    1312      
                                                                 
 max_pooling3d (MaxPooling3D  (None, 14, 49, 49, 16)   0         
 )                                                               
                                                                 
 conv3d_1 (Conv3D)           (None, 12, 47, 47, 32)    13856     
                                                                 
 max_pooling3d_1 (MaxPooling  (None, 6, 23, 23, 32)    0         
 3D)                                                             
                                                                 
 flatten (Flatten)           (None, 101568)            0         
                                                                 
 dense (Dense)               (None, 64)                6

/usr/local/lib/python3.8/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [9]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [10]:
def getFilePath(model_name):
    curr_dt_tm = datetime.datetime.now()
    return model_name + '_' + str(curr_dt_tm).replace(' ','').replace(':','_') + '.h5'

In [11]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
#filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.keras'
filepath = getFilePath(model_name)

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto',save_freq = num_epochs)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.001, verbose=1)# write the REducelronplateau code here
#callbacks_list = [checkpoint, LR]

In [12]:
def createModelCheckPoint(model_name):
    filepath = getFilePath(model_name)
    return ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto',save_freq = num_epochs)

def getCallbacksList(model_name):
    return [createModelCheckPoint(model_name), LR]

The `steps_per_epoch` and `validation_steps` are used by `fit` method to decide the number of next() calls it need to make.

In [13]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [44]:
model1.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=5, verbose=1, 
                    callbacks=getCallbacksList(model_name), validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  /home/datasets/Project_data/train ; batch size = 32
Epoch 1/5
 9/21 [===========>..................] - ETA: 1:28 - loss: 802.0500 - categorical_accuracy: 0.1979 - auc_2: 0.4969
Epoch 00001: saving model to model_init_2024-09-0304_47_20.471684/_2024-09-0304_52_02.922229.h5
19/21 [==========================>...] - ETA: 15s - loss: 380.9043 - categorical_accuracy: 0.2253 - auc_2: 0.5257
Epoch 00001: saving model to model_init_2024-09-0304_47_20.471684/_2024-09-0304_52_02.922229.h5
21/21 [==============================] - 190s 9s/step - loss: 349.4261 - categorical_accuracy: 0.2474 - auc_2: 0.5355 - val_loss: 1.4577 - val_categorical_accuracy: 0.3900 - val_auc_2: 0.6883 - lr: 0.0010
Epoch 2/5
 8/21 [==========>...................] - ETA: 1:41 - loss: 1.3467 - categorical_accuracy: 0.4844 - auc_2: 0.7693
Epoch 00002: saving model to model_init_2024-09-0304_47_20.471684/_2024-09-0304_52_02.922229.h5
18/21 [========================>.....] - ETA: 22s - loss: 1.3924 - categorical

**Observations**: The model is clearly overfitting with training accuracy of ~0.8 and validation accuracy of ~0.35.

#### Model 2: CNN + RNN

In [14]:
from tensorflow.keras.layers import SimpleRNN, TimeDistributed

input_shape = (len(img_idx), img_size[0], img_size[1], 3)

#write your model here
model2 = Sequential()
model2.add(Conv3D(16, kernel_size=(3,3,3), activation='relu', input_shape=input_shape))
model2.add(MaxPooling3D(pool_size=(2, 2, 2)))

model2.add(Conv3D(32, kernel_size=(3,3,3), activation='relu'))
model2.add(MaxPooling3D(pool_size=(2, 2, 2)))

model2.add(TimeDistributed(Flatten())) 

model2.add(SimpleRNN(64, activation='relu'))

model2.add(Dense(64, activation='relu'))
model2.add(Dense(5, activation='softmax'))

model2.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy', keras.metrics.AUC()])
model2.summary()

train_generator2 = generator(train_path, train_doc, batch_size)
val_generator2 = generator(val_path, val_doc, batch_size)


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d_2 (Conv3D)           (None, 28, 98, 98, 16)    1312      
                                                                 
 max_pooling3d_2 (MaxPooling  (None, 14, 49, 49, 16)   0         
 3D)                                                             
                                                                 
 conv3d_3 (Conv3D)           (None, 12, 47, 47, 32)    13856     
                                                                 
 max_pooling3d_3 (MaxPooling  (None, 6, 23, 23, 32)    0         
 3D)                                                             
                                                                 
 time_distributed (TimeDistr  (None, 6, 16928)         0         
 ibuted)                                                         
                                                      

In [49]:
model2.fit(train_generator2, steps_per_epoch=steps_per_epoch, epochs=5, verbose=1, 
                    callbacks=getCallbacksList(model_name), validation_data=val_generator2, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  /home/datasets/Project_data/train ; batch size = 32
Epoch 1/5
 9/21 [===========>..................] - ETA: 1:30 - loss: 69.8866 - categorical_accuracy: 0.2188 - auc_4: 0.5209
Epoch 00001: saving model to model_init_2024-09-0304_47_20.471684/_2024-09-0305_16_40.919276.h5
19/21 [==========================>...] - ETA: 15s - loss: 33.9474 - categorical_accuracy: 0.2171 - auc_4: 0.5249
Epoch 00001: saving model to model_init_2024-09-0304_47_20.471684/_2024-09-0305_16_40.919276.h5
21/21 [==============================] - 192s 10s/step - loss: 31.2647 - categorical_accuracy: 0.2142 - auc_4: 0.5246 - val_loss: 1.6095 - val_categorical_accuracy: 0.2300 - val_auc_4: 0.5086 - lr: 0.0010
Epoch 2/5
 8/21 [==========>...................] - ETA: 1:41 - loss: 1.5812 - categorical_accuracy: 0.2734 - auc_4: 0.5767
Epoch 00002: saving model to model_init_2024-09-0304_47_20.471684/_2024-09-0305_16_40.919276.h5
18/21 [========================>.....] - ETA: 23s - loss: 1.6073 - categorical_a

**Observations**: The model is not learning well with Categorical Accuracy: ~0.25-0.3 and Validation Categorical Accuracy: ~0.25. Even after 5 Epochs, based on AUC ~0.55-0.6, the model is only doing slightly better than a random guess.

#### Model 3: CNN + GRU

In [15]:
input_shape = (len(img_idx), img_size[0], img_size[1], 3)

#write your model here
model3 = Sequential()
model3.add(Conv3D(16, kernel_size=(3,3,3), activation='relu', input_shape=input_shape))
model3.add(MaxPooling3D(pool_size=(2, 2, 2)))

model3.add(Conv3D(32, kernel_size=(3,3,3), activation='relu'))
model3.add(MaxPooling3D(pool_size=(2, 2, 2)))

model3.add(TimeDistributed(Flatten())) 

model3.add(GRU(128, activation='tanh', return_sequences=False))

model3.add(Dense(64, activation='relu'))
model3.add(Dense(5, activation='softmax'))

model3.compile(optimizer=optimizers.Adam(lr=0.001, decay=1e-6), loss='categorical_crossentropy', metrics=['categorical_accuracy', keras.metrics.AUC()])
model3.summary()

train_generator3 = generator(train_path, train_doc, batch_size)
val_generator3 = generator(val_path, val_doc, batch_size)



Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d_4 (Conv3D)           (None, 28, 98, 98, 16)    1312      
                                                                 
 max_pooling3d_4 (MaxPooling  (None, 14, 49, 49, 16)   0         
 3D)                                                             
                                                                 
 conv3d_5 (Conv3D)           (None, 12, 47, 47, 32)    13856     
                                                                 
 max_pooling3d_5 (MaxPooling  (None, 6, 23, 23, 32)    0         
 3D)                                                             
                                                                 
 time_distributed_1 (TimeDis  (None, 6, 16928)         0         
 tributed)                                                       
                                                      

In [51]:
model3.fit(train_generator3, steps_per_epoch=steps_per_epoch, epochs=5, verbose=1, 
                    callbacks=getCallbacksList(model_name), validation_data=val_generator3, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  /home/datasets/Project_data/train ; batch size = 32
Epoch 1/5
 9/21 [===========>..................] - ETA: 1:34 - loss: 1.7887 - categorical_accuracy: 0.2083 - auc_5: 0.5006
Epoch 00001: saving model to model_init_2024-09-0304_47_20.471684/_2024-09-0305_34_29.140399.h5
19/21 [==========================>...] - ETA: 15s - loss: 1.7077 - categorical_accuracy: 0.1990 - auc_5: 0.4945
Epoch 00001: saving model to model_init_2024-09-0304_47_20.471684/_2024-09-0305_34_29.140399.h5
21/21 [==============================] - 193s 10s/step - loss: 1.7008 - categorical_accuracy: 0.1991 - auc_5: 0.4934 - val_loss: 1.6077 - val_categorical_accuracy: 0.2100 - val_auc_5: 0.5300 - lr: 0.0010
Epoch 2/5
 8/21 [==========>...................] - ETA: 1:43 - loss: 1.6121 - categorical_accuracy: 0.2344 - auc_5: 0.4940
Epoch 00002: saving model to model_init_2024-09-0304_47_20.471684/_2024-09-0305_34_29.140399.h5
18/21 [========================>.....] - ETA: 23s - loss: 1.6308 - categorical_accu

**Observations**: The model is not learning well with Categorical Accuracy: ~0.2 and Validation Categorical Accuracy: ~0.2. Even after 5 Epochs, based on AUC ~0.45-0.5, the model is only doing slightly better than a random guess.

#### Model 4: CNN + LSTM

In [16]:
from tensorflow.keras.layers import LSTM

input_shape = (len(img_idx), img_size[0], img_size[1], 3)

#write your model here
model4 = Sequential()
model4.add(Conv3D(16, kernel_size=(3,3,3), activation='relu', input_shape=input_shape))
model4.add(MaxPooling3D(pool_size=(2, 2, 2)))

model4.add(Conv3D(32, kernel_size=(3,3,3), activation='relu'))
model4.add(MaxPooling3D(pool_size=(2, 2, 2)))

model4.add(TimeDistributed(Flatten()))

model4.add(LSTM(128, activation='relu', return_sequences=False))

model4.add(Dense(64, activation='relu'))
model4.add(Dense(5, activation='softmax'))

model4.compile(optimizer=optimizers.Adam(lr=0.001, decay=1e-6), loss='categorical_crossentropy', metrics=['categorical_accuracy', keras.metrics.AUC()])
model4.summary()

train_generator4 = generator(train_path, train_doc, batch_size)
val_generator4 = generator(val_path, val_doc, batch_size)


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d_6 (Conv3D)           (None, 28, 98, 98, 16)    1312      
                                                                 
 max_pooling3d_6 (MaxPooling  (None, 14, 49, 49, 16)   0         
 3D)                                                             
                                                                 
 conv3d_7 (Conv3D)           (None, 12, 47, 47, 32)    13856     
                                                                 
 max_pooling3d_7 (MaxPooling  (None, 6, 23, 23, 32)    0         
 3D)                                                             
                                                                 
 time_distributed_2 (TimeDis  (None, 6, 16928)         0         
 tributed)                                                       
                                                      

In [17]:
callbacks_list4 = [createModelCheckPoint(model_name)]
model4.fit(train_generator4, steps_per_epoch=steps_per_epoch, epochs=5, verbose=1, 
                    callbacks=getCallbacksList(model_name), validation_data=val_generator4, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  /home/datasets/Project_data/train ; batch size = 32
Epoch 1/5


2024-09-03 07:04:11.451945: I tensorflow/stream_executor/cuda/cuda_dnn.cc:377] Loaded cuDNN version 8302


 9/21 [===========>..................] - ETA: 1:42 - loss: 366.9183 - categorical_accuracy: 0.2535 - auc_3: 0.5343
Epoch 00001: saving model to model_init_2024-09-0307_03_00.027498/_2024-09-0307_04_00.214070.h5
19/21 [==========================>...] - ETA: 17s - loss: 234.7911 - categorical_accuracy: 0.2434 - auc_3: 0.5297
Epoch 00001: saving model to model_init_2024-09-0307_03_00.027498/_2024-09-0307_04_00.214070.h5
21/21 [==============================] - 223s 11s/step - loss: 219.0838 - categorical_accuracy: 0.2489 - auc_3: 0.5326 - val_loss: 40.6901 - val_categorical_accuracy: 0.3700 - val_auc_3: 0.6201 - lr: 0.0010
Epoch 2/5
 8/21 [==========>...................] - ETA: 1:50 - loss: 33.7390 - categorical_accuracy: 0.4062 - auc_3: 0.6334
Epoch 00002: saving model to model_init_2024-09-0307_03_00.027498/_2024-09-0307_04_00.214070.h5
18/21 [========================>.....] - ETA: 25s - loss: 23.5067 - categorical_accuracy: 0.3715 - auc_3: 0.6337
Epoch 00002: saving model to model_init

**Observations**: With Categorical Accuracy: ~0.8 and Validation Categorical Accuracy: ~0.5-0.6 in 5 Epochs, the model is clearly overfitting. But still, it is better than all earlier models. The validation accuracy is steadily rising.

#### Model 5: CNN + LSTM (include Dropouts)

In [21]:
input_shape = (len(img_idx), img_size[0], img_size[1], 3)

#write your model here
model5 = Sequential()
model5.add(Conv3D(16, kernel_size=(3,3,3), activation='relu', input_shape=input_shape))
model5.add(MaxPooling3D(pool_size=(2, 2, 2)))
model5.add(TimeDistributed(Dropout(0.25)))

model5.add(Conv3D(32, kernel_size=(3,3,3), activation='relu'))
model5.add(MaxPooling3D(pool_size=(2, 2, 2)))
model5.add(TimeDistributed(Dropout(0.25)))

model5.add(TimeDistributed(Flatten()))

model5.add(LSTM(128, activation='relu', return_sequences=False))

model5.add(Dense(64, activation='relu'))
model5.add(Dropout(0.25))
model5.add(Dense(5, activation='softmax'))

model5.compile(optimizer=optimizers.Adam(lr=0.001, decay=1e-6), loss='categorical_crossentropy', metrics=['categorical_accuracy', keras.metrics.AUC()])
model5.summary()

train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d_12 (Conv3D)          (None, 28, 98, 98, 16)    1312      
                                                                 
 max_pooling3d_12 (MaxPoolin  (None, 14, 49, 49, 16)   0         
 g3D)                                                            
                                                                 
 time_distributed_9 (TimeDis  (None, 14, 49, 49, 16)   0         
 tributed)                                                       
                                                                 
 conv3d_13 (Conv3D)          (None, 12, 47, 47, 32)    13856     
                                                                 
 max_pooling3d_13 (MaxPoolin  (None, 6, 23, 23, 32)    0         
 g3D)                                                            
                                                      

/usr/local/lib/python3.8/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [22]:
model5.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=5, verbose=1, 
                    callbacks=getCallbacksList(model_name), validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  /home/datasets/Project_data/train ; batch size = 32
Epoch 1/5
 9/21 [===========>..................] - ETA: 1:32 - loss: 877.7565 - categorical_accuracy: 0.2188 - auc_6: 0.5125
Epoch 00001: saving model to model_init_2024-09-0307_03_00.027498/_2024-09-0307_40_40.299991.h5
19/21 [==========================>...] - ETA: 15s - loss: 637.9708 - categorical_accuracy: 0.2188 - auc_6: 0.5124
Epoch 00001: saving model to model_init_2024-09-0307_03_00.027498/_2024-09-0307_40_40.299991.h5
21/21 [==============================] - 191s 9s/step - loss: 593.0806 - categorical_accuracy: 0.2232 - auc_6: 0.5153 - val_loss: 12.5904 - val_categorical_accuracy: 0.2800 - val_auc_6: 0.5501 - lr: 0.0010
Epoch 2/5
 8/21 [==========>...................] - ETA: 1:39 - loss: 43.3515 - categorical_accuracy: 0.2266 - auc_6: 0.5231
Epoch 00002: saving model to model_init_2024-09-0307_03_00.027498/_2024-09-0307_40_40.299991.h5
16/21 [=====================>........] - ETA: 38s - loss: 24.2888 - categori

**Observations**: The model is clearly overfitting more than the Model #4

#### Model 6: CNN + LSTM (include Dropouts and BatchNormalization)

In [19]:
input_shape = (len(img_idx), img_size[0], img_size[1], 3)

#write your model here
model6 = Sequential()
model6.add(Conv3D(16, kernel_size=(3,3,3), activation='relu', input_shape=input_shape))
model6.add(MaxPooling3D(pool_size=(2, 2, 2)))
model6.add(TimeDistributed(Dropout(0.25)))

model6.add(Conv3D(32, kernel_size=(3,3,3), activation='relu'))
model6.add(MaxPooling3D(pool_size=(2, 2, 2)))
model6.add(TimeDistributed(Dropout(0.25)))

#model6.add(TimeDistributed(BatchNormalization()))
#model6.add(TimeDistributed(Dropout(0.1)))

model6.add(TimeDistributed(Flatten()))

model6.add(LSTM(128, activation='relu', return_sequences=False))

model6.add(Dense(64, activation='relu'))
model6.add(Dropout(0.1))
model6.add(BatchNormalization())

model6.add(Dense(5, activation='softmax'))

model6.compile(optimizer=optimizers.Adam(lr=0.001, decay=1e-6), loss='categorical_crossentropy', metrics=['categorical_accuracy', keras.metrics.AUC()])
model6.summary()

train_generator6 = generator(train_path, train_doc, batch_size)
val_generator6 = generator(val_path, val_doc, batch_size)

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d_10 (Conv3D)          (None, 28, 98, 98, 16)    1312      
                                                                 
 max_pooling3d_10 (MaxPoolin  (None, 14, 49, 49, 16)   0         
 g3D)                                                            
                                                                 
 time_distributed_6 (TimeDis  (None, 14, 49, 49, 16)   0         
 tributed)                                                       
                                                                 
 conv3d_11 (Conv3D)          (None, 12, 47, 47, 32)    13856     
                                                                 
 max_pooling3d_11 (MaxPoolin  (None, 6, 23, 23, 32)    0         
 g3D)                                                            
                                                      

In [20]:
model6.fit(train_generator6, steps_per_epoch=steps_per_epoch, epochs=5, verbose=1, 
                    callbacks=getCallbacksList(model_name), validation_data=val_generator6, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  /home/datasets/Project_data/train ; batch size = 32
Epoch 1/5
 9/21 [===========>..................] - ETA: 1:34 - loss: 2.1006 - categorical_accuracy: 0.1701 - auc_5: 0.4487
Epoch 00001: saving model to model_init_2024-09-0307_03_00.027498/_2024-09-0307_24_34.938222.h5
19/21 [==========================>...] - ETA: 15s - loss: 2.0223 - categorical_accuracy: 0.1612 - auc_5: 0.4530
Epoch 00001: saving model to model_init_2024-09-0307_03_00.027498/_2024-09-0307_24_34.938222.h5
21/21 [==============================] - 196s 10s/step - loss: 2.0187 - categorical_accuracy: 0.1614 - auc_5: 0.4518 - val_loss: 3.8530 - val_categorical_accuracy: 0.2100 - val_auc_5: 0.5289 - lr: 0.0010
Epoch 2/5
 8/21 [==========>...................] - ETA: 1:38 - loss: 1.8002 - categorical_accuracy: 0.1758 - auc_5: 0.5030
Epoch 00002: saving model to model_init_2024-09-0307_03_00.027498/_2024-09-0307_24_34.938222.h5
18/21 [========================>.....] - ETA: 23s - loss: 1.7406 - categorical_accu

**Observations**: The model is overfitting. The validation accuracy is not steadily rising and the AUC of 0.6 is not good either.

#### Model 7: CNN + LSTM (different dropouts)

In [23]:
input_shape = (len(img_idx), img_size[0], img_size[1], 3)

#write your model here
model7 = Sequential()
model7.add(Conv3D(16, kernel_size=(3,3,3), activation='relu', input_shape=input_shape))
model7.add(TimeDistributed(BatchNormalization()))
model7.add(MaxPooling3D(pool_size=(2, 2, 2)))

model7.add(Conv3D(32, kernel_size=(3,3,3), activation='relu'))
model7.add(TimeDistributed(BatchNormalization()))
model7.add(MaxPooling3D(pool_size=(2, 2, 2)))

model7.add(TimeDistributed(Flatten()))

model7.add(LSTM(128, activation='relu', return_sequences=False))
model7.add(Dropout(0.1))

model7.add(Dense(64, activation='relu'))

model7.add(Dense(5, activation='softmax'))

model7.compile(optimizer=optimizers.Adam(lr=0.001, decay=1e-6), loss='categorical_crossentropy', metrics=['categorical_accuracy', keras.metrics.AUC()])
model7.summary()

train_generator7 = generator(train_path, train_doc, batch_size)
val_generator7 = generator(val_path, val_doc, batch_size)


Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d_14 (Conv3D)          (None, 28, 98, 98, 16)    1312      
                                                                 
 time_distributed_12 (TimeDi  (None, 28, 98, 98, 16)   64        
 stributed)                                                      
                                                                 
 max_pooling3d_14 (MaxPoolin  (None, 14, 49, 49, 16)   0         
 g3D)                                                            
                                                                 
 conv3d_15 (Conv3D)          (None, 12, 47, 47, 32)    13856     
                                                                 
 time_distributed_13 (TimeDi  (None, 12, 47, 47, 32)   128       
 stributed)                                                      
                                                      

In [24]:
callbacks_list7 = [createModelCheckPoint(model_name)]
model7.fit(train_generator7, steps_per_epoch=steps_per_epoch, epochs=5, verbose=1, 
                    callbacks=getCallbacksList(model_name), validation_data=val_generator7, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  /home/datasets/Project_data/train ; batch size = 32
Epoch 1/5
 9/21 [===========>..................] - ETA: 1:28 - loss: 11.1451 - categorical_accuracy: 0.3715 - auc_7: 0.6380
Epoch 00001: saving model to model_init_2024-09-0307_03_00.027498/_2024-09-0308_04_59.058170.h5
19/21 [==========================>...] - ETA: 15s - loss: 8.5232 - categorical_accuracy: 0.3997 - auc_7: 0.6670
Epoch 00001: saving model to model_init_2024-09-0307_03_00.027498/_2024-09-0308_04_59.058170.h5
21/21 [==============================] - 193s 10s/step - loss: 8.2947 - categorical_accuracy: 0.4012 - auc_7: 0.6716 - val_loss: 65.6183 - val_categorical_accuracy: 0.2400 - val_auc_7: 0.5395 - lr: 0.0010
Epoch 2/5
 8/21 [==========>...................] - ETA: 1:36 - loss: 3.3596 - categorical_accuracy: 0.4609 - auc_7: 0.7622
Epoch 00002: saving model to model_init_2024-09-0307_03_00.027498/_2024-09-0308_04_59.058170.h5
18/21 [========================>.....] - ETA: 23s - loss: 3.7113 - categorical_ac

**Observations**: The model overfitting. In term of validation, eEven after 5 Epochs, the model is doing slightly well than a random guess.

#### Model 8: LSTM with Batch Normalization and extra convolution layers

In [27]:
input_shape = (len(img_idx), img_size[0], img_size[1], 3)

#write your model here
model8 = Sequential()
model8.add(Conv3D(16, kernel_size=(3,3,3), activation='relu', input_shape=input_shape))
model8.add(TimeDistributed(BatchNormalization()))
model8.add(MaxPooling3D(pool_size=(2, 2, 2)))

model8.add(Conv3D(32, kernel_size=(3,3,3), activation='relu'))
model8.add(TimeDistributed(BatchNormalization()))
model8.add(MaxPooling3D(pool_size=(2, 2, 2)))

model8.add(Conv3D(64, kernel_size=(3,3,3), activation='relu'))
model8.add(TimeDistributed(BatchNormalization()))
model8.add(MaxPooling3D(pool_size=(2, 2, 2), padding='same'))

model8.add(Conv3D(128, kernel_size=(3,3,3), activation='relu', padding='same'))
model8.add(TimeDistributed(BatchNormalization()))
model8.add(MaxPooling3D(pool_size=(2, 2, 2), padding='same'))

model8.add(TimeDistributed(Flatten()))

model8.add(LSTM(128, activation='relu', return_sequences=False))
model8.add(Dropout(0.25))

model8.add(Dense(64, activation='relu'))
model8.add(Dropout(0.25))

model8.add(Dense(5, activation='softmax'))

model8.compile(optimizer=optimizers.Adam(lr=0.001, decay=1e-6), loss='categorical_crossentropy', metrics=['categorical_accuracy', keras.metrics.AUC()])
model8.summary()

train_generator8 = generator(train_path, train_doc, batch_size)
val_generator8 = generator(val_path, val_doc, batch_size)


Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d_20 (Conv3D)          (None, 28, 98, 98, 16)    1312      
                                                                 
 time_distributed_20 (TimeDi  (None, 28, 98, 98, 16)   64        
 stributed)                                                      
                                                                 
 max_pooling3d_20 (MaxPoolin  (None, 14, 49, 49, 16)   0         
 g3D)                                                            
                                                                 
 conv3d_21 (Conv3D)          (None, 12, 47, 47, 32)    13856     
                                                                 
 time_distributed_21 (TimeDi  (None, 12, 47, 47, 32)   128       
 stributed)                                                      
                                                      

In [28]:
model8.fit(train_generator8, steps_per_epoch=steps_per_epoch, epochs=5, verbose=1, 
                    callbacks=getCallbacksList(model_name), validation_data=val_generator8,
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  /home/datasets/Project_data/train ; batch size = 32
Epoch 1/5
 9/21 [===========>..................] - ETA: 1:32 - loss: 2.2185 - categorical_accuracy: 0.3021 - auc_9: 0.6569
Epoch 00001: saving model to model_init_2024-09-0307_03_00.027498/_2024-09-0308_44_00.470019.h5
19/21 [==========================>...] - ETA: 15s - loss: 1.8734 - categorical_accuracy: 0.3618 - auc_9: 0.6842
Epoch 00001: saving model to model_init_2024-09-0307_03_00.027498/_2024-09-0308_44_00.470019.h5
21/21 [==============================] - 191s 9s/step - loss: 1.8200 - categorical_accuracy: 0.3680 - auc_9: 0.6942 - val_loss: 45.5295 - val_categorical_accuracy: 0.2200 - val_auc_9: 0.5125 - lr: 0.0010
Epoch 2/5
 8/21 [==========>...................] - ETA: 1:45 - loss: 1.5315 - categorical_accuracy: 0.4336 - auc_9: 0.7751
Epoch 00002: saving model to model_init_2024-09-0307_03_00.027498/_2024-09-0308_44_00.470019.h5
18/21 [========================>.....] - ETA: 23s - loss: 1.4374 - categorical_accu

**Observations**: The model is overfitting. The validation accuracy is not steadily rising and the validation AUC of 0.65 is not good either. But the validation accuracy is steadily rising.

#### Model 9: CNN + LSTM (with 25 Epochs)

In [17]:
from tensorflow.keras.layers import LSTM

input_shape = (len(img_idx), img_size[0], img_size[1], 3)

#write your model here
model9 = Sequential()
model9.add(Conv3D(16, kernel_size=(3,3,3), activation='relu', input_shape=input_shape))
model9.add(MaxPooling3D(pool_size=(2, 2, 2)))

model9.add(Conv3D(32, kernel_size=(3,3,3), activation='relu'))
model9.add(MaxPooling3D(pool_size=(2, 2, 2)))

model9.add(TimeDistributed(Flatten()))

model9.add(LSTM(128, activation='relu', return_sequences=False))

model9.add(Dense(64, activation='relu'))
model9.add(Dense(5, activation='softmax'))

model9.compile(optimizer=optimizers.Adam(lr=0.001, decay=1e-6), loss='categorical_crossentropy', metrics=['categorical_accuracy', keras.metrics.AUC()])
model9.summary()

train_generator9 = generator(train_path, train_doc, batch_size)
val_generator9 = generator(val_path, val_doc, batch_size)


Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d_8 (Conv3D)           (None, 28, 98, 98, 16)    1312      
                                                                 
 max_pooling3d_8 (MaxPooling  (None, 14, 49, 49, 16)   0         
 3D)                                                             
                                                                 
 conv3d_9 (Conv3D)           (None, 12, 47, 47, 32)    13856     
                                                                 
 max_pooling3d_9 (MaxPooling  (None, 6, 23, 23, 32)    0         
 3D)                                                             
                                                                 
 time_distributed_3 (TimeDis  (None, 6, 16928)         0         
 tributed)                                                       
                                                      

In [19]:
model9.fit(train_generator9, steps_per_epoch=steps_per_epoch, epochs=25, verbose=1, 
                    callbacks=getCallbacksList(model_name), validation_data=val_generator9, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  /home/datasets/Project_data/train ; batch size = 32
Epoch 1/25


2024-09-03 11:10:29.009127: I tensorflow/stream_executor/cuda/cuda_dnn.cc:377] Loaded cuDNN version 8302


 9/21 [===========>..................] - ETA: 1:32 - loss: 311.1915 - categorical_accuracy: 0.2014 - auc_4: 0.5060
Epoch 00001: saving model to model_init_2024-09-0311_04_42.748306/_2024-09-0311_10_18.429137.h5
19/21 [==========================>...] - ETA: 16s - loss: 162.3830 - categorical_accuracy: 0.2484 - auc_4: 0.5368
Epoch 00001: saving model to model_init_2024-09-0311_04_42.748306/_2024-09-0311_10_18.429137.h5
21/21 [==============================] - 207s 10s/step - loss: 149.7681 - categorical_accuracy: 0.2534 - auc_4: 0.5393 - val_loss: 6.9573 - val_categorical_accuracy: 0.2800 - val_auc_4: 0.5579 - lr: 0.0010
Epoch 2/25
 8/21 [==========>...................] - ETA: 1:48 - loss: 4.9509 - categorical_accuracy: 0.3281 - auc_4: 0.6135
Epoch 00002: saving model to model_init_2024-09-0311_04_42.748306/_2024-09-0311_10_18.429137.h5
18/21 [========================>.....] - ETA: 24s - loss: 4.1852 - categorical_accuracy: 0.3264 - auc_4: 0.6253
Epoch 00002: saving model to model_init_2

**Observations**: This model is same as Model #4 but with more Epochs. This was tried because Model #4 seemed the most promising with 5 Epochs. However, we can see that the validation categorical accuracy is stuck at 0.52-0.54 for more than 10 Epochs.

#### Model 10: Model 8 + 25 Epochs

In [21]:
input_shape = (len(img_idx), img_size[0], img_size[1], 3)

#write your model here
model10 = Sequential()
model10.add(Conv3D(16, kernel_size=(3,3,3), activation='relu', input_shape=input_shape))
model10.add(TimeDistributed(BatchNormalization()))
model10.add(MaxPooling3D(pool_size=(2, 2, 2)))

model10.add(Conv3D(32, kernel_size=(3,3,3), activation='relu'))
model10.add(TimeDistributed(BatchNormalization()))
model10.add(MaxPooling3D(pool_size=(2, 2, 2)))

model10.add(Conv3D(64, kernel_size=(3,3,3), activation='relu'))
model10.add(TimeDistributed(BatchNormalization()))
model10.add(MaxPooling3D(pool_size=(2, 2, 2), padding='same'))

model10.add(Conv3D(128, kernel_size=(3,3,3), activation='relu', padding='same'))
model10.add(TimeDistributed(BatchNormalization()))
model10.add(MaxPooling3D(pool_size=(2, 2, 2), padding='same'))

model10.add(TimeDistributed(Flatten()))

model10.add(LSTM(128, activation='relu', return_sequences=False))
model10.add(Dropout(0.25))

model10.add(Dense(64, activation='relu'))
model10.add(Dropout(0.25))

model10.add(Dense(5, activation='softmax'))

model10.compile(optimizer=optimizers.Adam(lr=0.001, decay=1e-6), loss='categorical_crossentropy', metrics=['categorical_accuracy', keras.metrics.AUC()])
model10.summary()

train_generator10 = generator(train_path, train_doc, batch_size)
val_generator10 = generator(val_path, val_doc, batch_size)


Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d_10 (Conv3D)          (None, 28, 98, 98, 16)    1312      
                                                                 
 time_distributed_4 (TimeDis  (None, 28, 98, 98, 16)   64        
 tributed)                                                       
                                                                 
 max_pooling3d_10 (MaxPoolin  (None, 14, 49, 49, 16)   0         
 g3D)                                                            
                                                                 
 conv3d_11 (Conv3D)          (None, 12, 47, 47, 32)    13856     
                                                                 
 time_distributed_5 (TimeDis  (None, 12, 47, 47, 32)   128       
 tributed)                                                       
                                                      

In [22]:
model10.fit(train_generator10, steps_per_epoch=steps_per_epoch, epochs=25, verbose=1, 
                    callbacks=getCallbacksList(model_name), validation_data=val_generator10,
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  /home/datasets/Project_data/train ; batch size = 32
Epoch 1/25
 9/21 [===========>..................] - ETA: 1:36 - loss: 1.8699 - categorical_accuracy: 0.3472 - auc_5: 0.6428
Epoch 00001: saving model to model_init_2024-09-0311_04_42.748306/_2024-09-0312_43_53.061035.h5
19/21 [==========================>...] - ETA: 15s - loss: 1.7063 - categorical_accuracy: 0.3931 - auc_5: 0.6972
Epoch 00001: saving model to model_init_2024-09-0311_04_42.748306/_2024-09-0312_43_53.061035.h5
21/21 [==============================] - 192s 10s/step - loss: 1.6901 - categorical_accuracy: 0.3937 - auc_5: 0.7004 - val_loss: 27.7157 - val_categorical_accuracy: 0.2400 - val_auc_5: 0.5212 - lr: 0.0010
Epoch 2/25
 8/21 [==========>...................] - ETA: 1:38 - loss: 1.2957 - categorical_accuracy: 0.4844 - auc_5: 0.7895
Epoch 00002: saving model to model_init_2024-09-0311_04_42.748306/_2024-09-0312_43_53.061035.h5
18/21 [========================>.....] - ETA: 23s - loss: 1.3142 - categorical_a

**Observations**: This model is same as Model #8 but with 25 Epochs. Even now the model is overfitting but not by a very big margin. 
**This is the final model.**